# Day 7 - Lab 2: Agent Interoperability with A2A Protocol

**Objective:** To provide students with hands-on experience implementing the A2A Protocol, enabling them to build two distinct agents that can discover and communicate with each other in a standardized way.

**Estimated Time:** 135 minutes

**Introduction:**
For agents to collaborate effectively without human intervention, they need a shared language—a standard protocol. The Agent2Agent (A2A) Protocol provides this standard, allowing agents to discover each other's capabilities and communicate securely. In this lab, you will build two separate agents: a 'Responder' that offers a service and a 'Requester' that consumes it. Finally, you will integrate this communication into a high-level LangChain agent, bridging the gap between low-level protocols and agentic reasoning.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will install the `a2a-protocol` library. This lab is unique because it requires running two separate Python scripts simultaneously. You will need to run the Responder agent in one terminal and the Requester agent in another.

**Model Selection:**
For the final challenge, where we integrate A2A into a LangChain agent, a model with strong tool-using capabilities like `gpt-4.1`, `o3`, or `gemini-2.5-pro` is recommended.

In [2]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

import importlib
def install_if_missing(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"{package} not found, installing...")
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_if_missing('a2a')

from utils import setup_llm_client, save_artifact
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

✅ LLM Client configured: Using 'openai' with model 'gpt-4o'


## Step 2: The Challenges

These challenges involve creating and running separate Python scripts. You will write the code in the cells below, and then save and execute them from your terminal.

### Challenge 1 (Foundational): Implementing the Responder Agent

**Task:** Create a simple 'Responder' agent that offers a service over the A2A protocol. This agent will act as the 'server' in our interaction.

**Instructions:**
1.  Create the code for a Python script named `a2a_responder.py`.
2.  Define a simple Python function, `calculate_task_complexity`, that takes two integer arguments (`steps` and `priority`) and returns a string.
3.  Import `Responder` and `Service` from `a2a_protocol`.
4.  Create a `Service` object, wrapping your `calculate_task_complexity` function.
5.  Instantiate the `Responder`, giving it a unique name and adding your service to it.
6.  Start the responder using `responder.start()`.
7.  Save this script to a file.

**To Run:** Open a terminal, navigate to your project directory, and run `python a2a_responder.py`. It should start up and wait for connections.

In [ ]:
responder_code = """# a2a_responder.py
import time
from a2a_protocol import Responder, Service

# TODO: 1. Define the function that will be our service.
def calculate_task_complexity(steps: int, priority: int) -> str:
    # Your logic here. For example, complexity = steps * priority
    # Return a descriptive string.
    return ""

def main():
    print("Initializing Responder Agent...")
    
    # TODO: 2. Create a Service object from the function.
    complexity_service = None # Your code here
    
    # TODO: 3. Instantiate the Responder.
    responder = None # Give it a name and register the service.
    
    try:
        # TODO: 4. Start the responder.
        print(f"Responder '{responder.name}' started at {responder.address}")
        # Keep the script alive to listen for requests
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("\nShutting down responder...")
        responder.stop()

if __name__ == "__main__":
    main()
"""

save_artifact(responder_code, "a2a_responder.py")
print("Saved 'a2a_responder.py'. Run it in a separate terminal.")

### Challenge 2 (Intermediate): Implementing the Requester Agent

**Task:** Create a 'Requester' agent that connects to the Responder, discovers its services, and invokes one.

**Instructions:**
1.  Create the code for a Python script named `a2a_requester.py`.
2.  Import `Requester` from `a2a_protocol`.
3.  Instantiate the `Requester`.
4.  Use `requester.discover_responders()` to find the running Responder agent.
5.  Connect to the first discovered responder and use `responder_proxy.discover_services()` to see what it offers.
6.  Invoke the `calculate_task_complexity` service with some sample data.
7.  Print the result.

**To Run:** In a *second* terminal (while the responder is still running), execute `python a2a_requester.py`.

In [ ]:
requester_code = """# a2a_requester.py
import asyncio
from a2a_protocol import Requester

async def main():
    print("Initializing Requester Agent...")
    # TODO: 1. Instantiate the Requester
    requester = None # Your code here

    print("Discovering responders on the network...")
    # TODO: 2. Discover responders. This may take a few seconds.
    responders = [] # Your code here

    if not responders:
        print("No responders found. Is the a2a_responder.py script running?")
        return

    # Connect to the first responder found
    responder_proxy = responders[0]
    print(f"Connected to responder: {responder_proxy.name}")

    # TODO: 3. Discover the services offered by the responder.
    services = [] # Your code here
    print(f"Discovered services: {services}")

    if 'calculate_task_complexity' in services:
        print("\nInvoking 'calculate_task_complexity' service...")
        # TODO: 4. Invoke the service with keyword arguments.
        result = None # Your code here
        print(f"Service responded with: {result}")
    else:
        print("The required service was not found.")

if __name__ == "__main__":
    asyncio.run(main())
"""

save_artifact(requester_code, "a2a_requester.py")
print("Saved 'a2a_requester.py'. Run it in a second terminal while the responder is running.")

### Challenge 3 (Advanced): Integrating A2A as a LangChain Tool

**Task:** Bridge the gap between the low-level A2A protocol and a high-level reasoning agent by wrapping the A2A client in a LangChain tool.

**Instructions:**
1.  Import the necessary LangChain components (`@tool`, `AgentExecutor`, etc.).
2.  Define a new function, `get_task_complexity`, that takes `steps` and `priority` as arguments.
3.  Inside this function, place the A2A Requester logic from Challenge 2. It should connect, discover, and invoke the service, returning the final result.
4.  Decorate this function with LangChain's `@tool` decorator and give it a descriptive docstring.
5.  Create a LangChain agent and provide it with this new tool.
6.  Invoke the agent with a high-level natural language prompt, like: `"How complex is a task with 8 steps and a priority level of 5?"`

**Expected Quality:** A seamless workflow where a high-level LangChain agent uses its reasoning to call a tool, which in turn uses the A2A protocol to communicate with another specialized agent to get the job done.

In [ ]:
# TODO: Write the code for the LangChain agent with the A2A-powered tool.
# This will be a single script/cell that you can run after starting the responder.


## Lab Conclusion

Excellent work! You have successfully implemented the Agent2Agent protocol, creating two distinct agents that can communicate in a standardized way. More importantly, you integrated this low-level communication into a high-level LangChain agent, demonstrating how specialized, protocol-driven agents can be exposed as tools for more general-purpose reasoning agents. This is a key architectural pattern for building complex, interoperable multi-agent systems.